In [1]:
import pandas as pd
import plotly.graph_objects as go
from signals import *

In [2]:
def analyse_signal(df, signal):
    """
    Process signal using SL and TP.
    TODO: accound for spread and fees
    TODO: Warn if slippage is high
    """
    real_entry = signal.entry_price
    
    for dt, ohlc in df.loc[signal.datetime_str:].iterrows():
        if signal.type == "BUY":
            if ohlc.High >= signal.take_profit:
                return signal.take_profit - real_entry

            if ohlc.Low <= signal.stop_loss:
                return signal.stop_loss - real_entry
            
        if signal.type == "SELL":
            if ohlc.Low <= signal.take_profit:
                return real_entry - signal.take_profit

            if ohlc.High >= signal.stop_loss:
                return real_entry - signal.stop_loss
        
    return 0.0
        

In [3]:
beta_tp1 = Signal(ticker="AUDCAD", datetime_str="2020-01-14 13:19", 
                  entry_price=0.9020, stop_loss=0.9075, take_profit=0.9000)
df = load_ticker(beta_tp1.ticker)
print("TP1", analyse_signal(df, beta_tp1) * pip_factor(beta_tp1.ticker))

TP1 20.000000000000018


In [33]:
# TP1 Analysis using trade signals as given
signals = load_beta_trades("./beta/BETA.csv")
total_profit = 0.0
wins = 0
losses = 0
for signal in signals["TP3"]:
    df = load_ticker(signal.ticker, 60)
    #signal.set_stop_loss_pips(24)
    #signal.set_take_profit_pips(70)
    
    profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
    if profit > 0:
        wins += 1
    else:
        losses += 1
    total_profit += profit
    print(signal, profit)
print("Default:", total_profit, wins, losses)

AUDCAD: 2020-01-14 13:19: Entry: 0.902, SL: 0.9075, TP: 0.892 0.0
AUDNZD: 2020-01-14 13:16: Entry: 1.043, SL: 1.048, TP: 1.033 0.0
CADJPY: 2020-01-14 13:13: Entry: 84.2, SL: 84.85, TP: 83.2 0.0
AUDUSD: 2020-01-14 13:08: Entry: 0.69, SL: 0.6965, TP: 0.68 0.0
NZDJPY: 2020-01-14 13:06: Entry: 72.8, SL: 73.6, TP: 71.8 0.0
XAUUSD: 2020-01-14 10:56: Entry: 1544.94, SL: 1534.0, TP: 1554.94 100.0
GBPUSD: 2020-01-13 14:06: Entry: 1.29843, SL: 1.29, TP: 1.30843 100.00000000000009
XAUUSD: 2020-01-10 01:31: Entry: 1551.91, SL: 1558.0, TP: 1541.91 -60.89999999999918
GBPJPY: 2020-01-09 02:08: Entry: 143.174, SL: 143.674, TP: 142.174 -50.0
USDCAD: 2020-01-06 13:03: Entry: 1.2969, SL: 1.293, TP: 1.3069 100.00000000000009
NZDCAD: 2019-12-04 12:44: Entry: 0.8656, SL: 0.8715, TP: 0.8556 -59.00000000000016
AUDJPY: 2019-12-04 12:43: Entry: 74.27, SL: 74.9, TP: 73.27 -63.000000000000966
CADJPY: 2019-12-04 12:41: Entry: 81.86, SL: 82.5, TP: 80.86 -64.00000000000006
NZDUSD: 2019-12-04 12:40: Entry: 0.6518, SL

In [20]:
# Given TP Stop loss analysis
signals = load_beta_trades("./beta/BETA.csv")

stops = range(1, 150, 1)
profits = []
take_profit = "TP3"

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals[take_profit]:
        signal.set_stop_loss_pips(stop)
        df = load_ticker(signal.ticker, 60)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    #print(stop, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title=f"BETA: Stop Loss Optimisation for {take_profit}",
    xaxis_title="Stop Loss (pips)",
    yaxis_title="Profit (pips)",
    title_x=0.5
)
fig.show()


In [29]:
# Take Profit analysis
signals = load_beta_trades("./beta/BETA.csv")

stop_loss = 25
take_profits = range(10, 100, 1)
profits = []

for take_profit in take_profits:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP3"]:
        signal.set_stop_loss_pips(stop_loss)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker, 60)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    #print(take_profit, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title=f"BETA: Take Profit Optimisation (Stop Loss = {stop_loss} pips)",
    title_x=0.5,
    xaxis_title="Take Profit (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()
    

In [28]:
# Stop Loss analysis
signals = load_beta_trades("./beta/BETA.csv")

take_profit = 70
stops = range(10, 100, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP3"]:
        signal.set_stop_loss_pips(stop)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker, 60)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    #print(stop, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title=f"BETA: Stop Loss Optimisation (Take Profit = {take_profit} pips)",
    title_x=0.5,
    xaxis_title="Stop Loss Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()
    

In [ ]:
# Example plotly candlestick plot

dfp = df.loc["2019-01-14 21:00":"2019-01-14 22:00"]
fig = go.Figure(data=[go.Candlestick(x=dfp.index,
                open=dfp['Open'],
                high=dfp['High'],
                low=dfp['Low'],
                close=dfp['Close'])])

fig.show()